In [1]:
import numpy as np
import tensorflow as tf

# data

In [2]:
# let's create a temporary variable npz, where we will store each of the three Audiobooks datasets
npz = np.load(r"C:\Users\Kevin's PC\Audiobooks_data_train.npz")

# we extract the inputs using the keyword under which we saved them
# to ensure that they are all floats, let's also take care of that
train_inputs = npz['inputs'].astype(np.float)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype(np.int)

# we load the validation data in the temporary variable
npz = np.load(r"C:\Users\Kevin's PC\Audiobooks_data_validation.npz")
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

# we load the test data in the temporary variable
npz = np.load(r"C:\Users\Kevin's PC\Audiobooks_data_test.npz")
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

# Model

In [6]:
# Set the input and output sizes
input_size = 10
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 20
    
# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


### Choose the optimizer and the loss function

# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
# That's where we train the model we have built.

# set the batch size
batch_size = 100

# set a maximum number of training epochs
max_epochs = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          callbacks=[early_stopping], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )  

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 1s - loss: 0.6302 - accuracy: 0.6787 - val_loss: 0.5523 - val_accuracy: 0.8255
Epoch 2/100
3579/3579 - 0s - loss: 0.4921 - accuracy: 0.8438 - val_loss: 0.4320 - val_accuracy: 0.8568
Epoch 3/100
3579/3579 - 0s - loss: 0.3954 - accuracy: 0.8692 - val_loss: 0.3625 - val_accuracy: 0.8635
Epoch 4/100
3579/3579 - 0s - loss: 0.3481 - accuracy: 0.8776 - val_loss: 0.3310 - val_accuracy: 0.8814
Epoch 5/100
3579/3579 - 0s - loss: 0.3256 - accuracy: 0.8807 - val_loss: 0.3122 - val_accuracy: 0.8881
Epoch 6/100
3579/3579 - 0s - loss: 0.3117 - accuracy: 0.8821 - val_loss: 0.3001 - val_accuracy: 0.8949
Epoch 7/100
3579/3579 - 0s - loss: 0.3007 - accuracy: 0.8866 - val_loss: 0.2890 - val_accuracy: 0.8926
Epoch 8/100
3579/3579 - 0s - loss: 0.2921 - accuracy: 0.8885 - val_loss: 0.2790 - val_accuracy: 0.8971
Epoch 9/100
3579/3579 - 0s - loss: 0.2860 - accuracy: 0.8902 - val_loss: 0.2731 - val_accuracy: 0.8949
Epoch 10/100
3579/3579 - 0

In [7]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

448/448 [==============================] - 0s 306us/sample - loss: 0.2364 - accuracy: 0.9129


In [8]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.24. Test accuracy: 91.29%
